In [35]:
import numpy as np
import pandas as pd

import keras
from sklearn.model_selection import train_test_split
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras import optimizers
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras import backend as K

keras.backend.set_image_data_format('channels_last')
print(f"Data format set to: {K.image_data_format()}")

print('Imported real good!')


Data format set to: channels_last
Imported real good!


In [2]:
df = pd.read_csv('data.csv', index_col=False)
df.head(1)

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,50


# Process the data

In [3]:
# Split labels from data
labels_unordered = np.array(df[['784']])

# Need labels used to be numbered correctly to call to_categorical()
labels = np.copy(labels_unordered)
labels = np.where(labels == 49, 10, labels)
labels = np.where(labels == 50, 11, labels)
labels = np.where(labels == 51, 12, labels)
labels = np.where(labels == 52, 13, labels)
labels = np.where(labels == 82, 14, labels)
labels = np.where(labels == 33, 15, labels)
labels = np.where(labels == 34, 16, labels)

# make sure we gucci
assert len(labels_unordered) == len(labels), print(
    'labels_fixd has diff num of elements')
assert len(np.unique(labels)) == len(
    np.unique(labels_unordered)), print('Diff num of unique')

# vectorize classes
num_labels = len(np.unique(labels))
cat = to_categorical(labels, num_labels)

# Drop label col from data
df.drop(columns=['784'], inplace=True)

# reshape data - Channel last format
train_data = df.values
# train_data = train_data.reshape(-1, 28, 28, 1)
# # there are 240520 images in the data set, each image is 28x28 pixels, and only 1 color channel
# assert train_data.shape == (240520, 28, 28, 1), print(
#     "training data has wrong dimensions")


In [33]:
l = []
for img in train_data:
    l.append(img.reshape(28,28,1))

In [4]:
# split into train test
data_train, data_test, label_train, label_test = train_test_split(train_data, 
                                                                  cat, test_size = 0.3, train_size=0.7, 
                                                                  random_state=42)

print(data_test.shape, data_train.shape, label_train.shape, label_test.shape)


(72156, 28, 28, 1) (168364, 28, 28, 1) (168364, 17) (72156, 17)


In [34]:
l = np.array(l)
l.shape

(240520, 28, 28, 1)

In [ ]:
l.shape

In [36]:
model = Sequential()
model.add(Conv2D(30, (5, 5), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(15, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(17, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [37]:
model.fit(np.array(l), cat, epochs=10, batch_size=200, shuffle=True, verbose=1)

Epoch 1/10
1203/1203 [==============================] - 81s 63ms/step - loss: 1.1432 - accuracy: 0.7262
Epoch 2/10
1203/1203 [==============================] - 72s 60ms/step - loss: 0.3018 - accuracy: 0.9001
Epoch 3/10
1203/1203 [==============================] - 92s 76ms/step - loss: 0.2383 - accuracy: 0.9191
Epoch 4/10
1203/1203 [==============================] - 98s 82ms/step - loss: 0.2105 - accuracy: 0.9277
Epoch 5/10
1203/1203 [==============================] - 75s 62ms/step - loss: 0.1913 - accuracy: 0.9344
Epoch 6/10
1203/1203 [==============================] - 74s 61ms/step - loss: 0.1781 - accuracy: 0.9386
Epoch 7/10
1203/1203 [==============================] - 74s 61ms/step - loss: 0.1691 - accuracy: 0.9418
Epoch 8/10
1203/1203 [==============================] - 75s 62ms/step - loss: 0.1609 - accuracy: 0.9445
Epoch 9/10
1203/1203 [==============================] - 74s 61ms/step - loss: 0.1552 - accuracy: 0.9467
Epoch 10/10
1203/1203 [==============================] - 74s 61m

In [38]:
from keras.models import model_from_json

model_json = model.to_json()
with open("basic_model.json", "w") as json_model:
    json_model.write(model_json)
    
model.save_weights("basic_model_weights.h5")